based on the paper __1995 StorckHochreiterSchmidhuber - Reinforcement driven information acquisition in nondeterministc environments__

In [27]:
import mdptoolbox, mdptoolbox.example

n_states = 2; n_actions = 3
P, R = mdptoolbox.example.rand(n_states, n_actions)
fh = mdptoolbox.mdp.FiniteHorizon(P, R, 0.9, 4)
fh.run()
print fh.V
print
print
print fh.policy
print
print
print P.shape
print P
print
print
print R.shape

[[ 0.65734076  0.97013582  0.36317169  0.74933843  0.        ]
 [ 0.44404809 -0.10221963  0.24533043 -0.42907415  0.        ]]


[[0 0 0 0]
 [0 0 0 0]]


(3, 2, 2)
[[[ 0.          1.        ]
  [ 1.          0.        ]]

 [[ 0.          1.        ]
  [ 0.24074908  0.75925092]]

 [[ 0.          1.        ]
  [ 1.          0.        ]]]


(3, 2, 2)


In [39]:
import numpy as np

 
def prob_ijk(at,St,Stplus,c_ijk):
    c_ij = np.sum(c_ijk[at,St,:])
    p_ijk = c_ijk[at,St,:]/c_ij
    
    if c_ij == 0:
        p_ijk = c_ijk[at,St,:]*0
    return p_ijk
    
def information_gain_D(at,St,Stplus,c_ijk):
    p_ijk_t = prob_ijk(at,St,Stplus,c_ijk)
    
    c_ijk[at,St,Stplus] += 1
    p_ijt_tplus = prob_ijk(at,St,Stplus,c_ijk)
    return sum(abs(p_ijt_tplus-p_ijk_t))    
    
def update_Q(Q,at,St,Stplus,c_ijk):
    alpha = .5
    gamma = .45
    D = information_gain_D(at,St,Stplus,c_ijk)
    maxQ = max(Q[:,Stplus])
    Q = (1-alpha)*Q[at,St] + alpha*(D+gamma*maxQ)      
    return Q

In [86]:
from itertools import product
from __future__ import division

terminal_time = 1000
epsilon = .5
Q = np.zeros((n_actions,n_states))
c_ijk = np.zeros((n_actions, n_states, n_states))
P_experimental = np.zeros((n_actions, n_states, n_states))

# start from a random place
St = np.random.randint(0, n_states)

count = 0
for t in np.arange(terminal_time):
    
    # 1. pick at
    # epsilon greedy action
    if np.random.rand() < epsilon:
        count += 1
        #print count/t
        at = np.random.randint(0, n_actions)
    else:
        at = np.argmax(Q[St,:])
    #print at
    
    # 2. execute at and figure out where you end up, in which S(t+1)
    Stplus = np.random.choice(n_states, 1, p=P[at,St,:])  
    #print St,Stplus
    
    # 3. update Q value
    Q[at,St] = update_Q(Q,at,St,Stplus,c_ijk)
    
    c_ijk[at,St,Stplus] += 1
    St = Stplus[0]

    
    
# construct the final experimental transition matrix    
for i,j in product(np.arange(n_actions), np.arange(n_states)):
    c_ij = np.sum(c_ijk[i,j,:])
    for k in np.arange(n_states):
        P_experimental[i,j,k] = c_ijk[i,j,k]/c_ij
        if c_ij == 0:
            P_experimental[i,j,k] = 0



print P
print
print
print P_experimental

[[[ 0.          1.        ]
  [ 1.          0.        ]]

 [[ 0.          1.        ]
  [ 0.24074908  0.75925092]]

 [[ 0.          1.        ]
  [ 1.          0.        ]]]


[[[ 0.          1.        ]
  [ 1.          0.        ]]

 [[ 0.          1.        ]
  [ 0.23923445  0.76076555]]

 [[ 0.          1.        ]
  [ 1.          0.        ]]]


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in divide
